<a href="https://colab.research.google.com/github/FabioDamacena/Pandas/blob/main/Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Uso básico do pandas**

In [1]:
import pandas as pd

In [ ]:
pd.__version__

## **A estrutura de um data frame**

In [75]:
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

In [ ]:
df.head()

In [ ]:
df.info()

## **Índice**

In [ ]:
df.index

Depois de lido o arquivo, é possível alterar o índice para ser o ID do passageiro, conforme comando abaixo:

In [58]:
df.set_index('PassengerId', inplace = True)

In [ ]:
df.head()

Características de um dataframe

In [ ]:
df.columns

In [ ]:
df.values

In [ ]:
type(df.values)

Dataframes vs séries: séries são mais limitadas, dataframes abrange toda uma estrutura de dados.
*   Métodos diferentes
*   N dimensões






Consultando dados (diferente do SQL). método loc possibilita consultar dados dentro do dataframe:

In [ ]:
df.loc[1] #consulta linha 1

In [ ]:
df.loc[[1, 2, 3]] #consulta várias linhas

In [ ]:
df.loc[10:20] # da posição 10 até 20

In [ ]:
df.loc[10:30:2] # posição 10 até 30 com passo de 2 em 2

In [ ]:
df.loc[10:] # do 10 até o último

Selecionando linhas e colunas com o loc

df.loc[[linha1, linha2], [coluna1, coluna2, coluna3]]

In [ ]:
df.loc[[1, 2], ['Name', 'Sex', 'Age']]

In [ ]:
df.loc[:, ['Name', 'Sex', 'Age']] #Seleciona todas as linhas e as colunas solicitadas

Método query: simplifica, traduz a consulta (iloc passa a posição que quer alterar):

In [ ]:
df.query('Age > 20').head()

In [ ]:
df.query('Age > 20 & Sex=="male"').head(10)

Usando o operador OR:

In [ ]:
df.query('Age > 20 | Sex=="Male"').head(10)

Operador IN e parâmetro inplace (altera os dados do dataframe):

In [50]:
df.query('Embarked in ["C", "Q"]', inplace=True)

In [ ]:
df.head()

filtrando dados tipo object:

In [ ]:
df.select_dtypes(include=['object'])

## **Analisando um Dataframe com profiling**

Ajuda a inspecionar um dataframe:

Ajuda a identificar missing values, zeros, high cardinality e distincts values

Estatísticas por coluna (quartis, missing, média, mínimo, máximo, distintos, zeros, histograma para ver a distribuição, range, interquartile range, estatísticas descritivas (coeficiente de variação, desvio padrão, soma, variância, média, mediana))

In [77]:
import pandas_profiling

In [78]:
try:
  pandas_profiling.ProfileReport(df)
except:
  print("Não foi possível executar o comando.")

Não foi possível executar o comando.


## **Agregação de dados**

Agrupa e conta a quantidade por sexo:

In [ ]:
df.groupby(by='Sex').size()

In [ ]:
df.Sex.value_counts() #faz a mesma coisa, porém o groupy possibilita extender a mais de uma coluna para agrupar.

In [ ]:
df.groupby(by='Sex').count()

Agrupando por sexo e calculando a média das idades:

In [ ]:
df.groupby(by='Sex')['Age'].mean()

Para trabalhar com os dados, usar index e values (transforme em lista, em vez de array numpy, para poder plotar gráficos e executar outros trabalhos):

In [67]:
s1 = df.groupby(by='Sex')['Age'].mean()

In [ ]:
s1.index.values.tolist()

In [ ]:
s1.values.tolist()

Média das idades das pessoas que sobreviveram e das que não sobreviveram:

Manda agrupar por sexo e se sobreviveu ou não, depois manda agregar pela media da idade e pela quantidade de passageiros.

In [71]:
import numpy as np

In [ ]:
df.groupby(['Sex', 'Survived']).agg({'Age': np.mean, 'PassengerId': np.size})

## **DataCleaning e Pré-Processamento de dados**

Gerando amostra aleatória dos dados:

In [ ]:
df.sample(n=50).head()

25% de dados aleatórios:

In [ ]:
df.sample(frac=.25, random_state=1).head()

Tratando missing values:

In [ ]:
df.isnull().sum()

Métodos de preenchimento de valores faltantes:

df.fillna(0) pega os valores faltantes e preenche com zero

df.fillna(df.Age.mean()) preenche com a média da coluna

df.fillna(method='ffill') pega o último valor anterior que não era NA e preenche

df.fillna(method='bfill') pega o último valor anterior que não era null e preenche

values = {'A': 0, 'B': 1, 'C': 2, 'D': 3} -> df.fillna(value=values) define dicionário de valores




In [91]:
values = {'Age': df.Age.mode()[0], 'Cabin': 'SC', 'Embarked': df.Embarked.mode()[0]} #preencheu idade e embarcado com a moda, cabin com SC (sem informação)

In [92]:
df.fillna(value=values, inplace=True)

Manipulando strings:

In [ ]:
df.Name.str.rstrip().head() #remove dado em branco da direita. lstrip da esquerda

In [ ]:
df.Name.str.lower().head()

In [ ]:
df.Name.str.upper().head()

Cleaning textos:

In [98]:
def remove_parenteses(item):
  if '(' in item:
    return item.replace('(','').replace(')','')
  else:
    return item

In [ ]:
df.Name.head(10).apply(remove_parenteses) #método applymap() pode ser usado a nível de dataframe (em todas as colunas de uma vez)

Trabalhando com expressão regular: (ex.: buscar todos os nomes que têm a expressão 'Mr' na frente) busca um padrão.

In [100]:
import re

In [ ]:
df.loc[df.Name.str.contains('Mr', flags=re.I, regex=True)].head()

Usando lambda para buscar quais linhas possuem Mr e qual a última letra do nome das linhas que contêm Mr:

In [102]:
def checa_nome(name, last_letter):
  if re.search('Mr', name):
    return last_letter
  else:
    return 0

In [103]:
df['Mr_name'] = df.Name.apply(lambda x: checa_nome(x, x[-1])) #criou nova coluna

In [ ]:
df.head(20)

# **Trabalhando com grande base de dados (aprimorando performance do pandas)**

1.   Tente trabalhar apenas com as colunas que realmente vai precisar
2.   Atentar para o tipo de dado de cada coluna
3.   Usar chunksize para dividir o processamento por quantidade de linhas

Dica: usar o comando head para ler as primeiras 5 primeiras linhas e verificar qual é o separador:

df = pd.read_csv("link.csv", sep=',', nrow=5)



## **Tente trabalhar apenas com as colunas que realmente vai precisar**

Exportar o nome das colunas para usar no parâmetro usecols (para o caso de dados com centenas de colunas, as quais não utilizará todas)

In [ ]:
df.columns.tolist()

In [107]:
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv", usecols=['PassengerId', 'Survived', 'Name', 'Sex'])

In [ ]:
df.head()

In [ ]:
df.info() #usa bem menos memória com menos colunas, o que faz diferença para grande número de dados

Ler todas as colunas, exceto algumas, usando lambda:

In [110]:
data = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(data, usecols = lambda column : column not in ["Parch", "Ticket", "Fare", "Cabin"])

In [ ]:
df.head()

In [ ]:
df.info()

## **Atentar para o tipo de dado de cada coluna**

In [113]:
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

In [ ]:
df.head()

In [ ]:
df.info()

Convertendo os tipos de dados:

In [117]:
df.Sex = df.Sex.astype('category')
df.Embarked = df.Embarked.astype('category')
df.Survived = df.Survived.astype('category')
df.Pclass = df.Pclass.astype('category')
df.PassengerId = df.PassengerId.astype('int32')
df.Parch = df.Parch.astype('int32')
df.SibSp = df.SibSp.astype('int32')

In [ ]:
df.info() #melhorou o uso de memória

Convertendo colunas em tempo de leitura:

In [119]:
data = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(data, dtype = {"Embarked": "category", "Survived": "category", "Parch": "int32"})

In [ ]:
df.info()

## **Usar chunksize**

Parâmetro também disponível no read_json

In [121]:
data = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"

In [ ]:
for d in pd.read_csv(data, chunksize=200):
  print("Chunk")
  print(d.count()) #divide em partes (diferentes dataframes com até 200 linhas)

In [125]:
age_mean = []
age_std = []
sex_dist = []
male_count = []
female_count = []

In [ ]:
try:
  for d in pd.read_csv(data, chunksize=200):
    age_mean.append(d.age_mean())
    age_std.append(d.Age.std())
    male_count.append(d.Sex.value_counts().values[0])
    female_count.append(d.Sex.value_counts().values[1])
except:
  print("Não foi possível executar o comando.")

In [ ]:
print(age_mean)
print(age_std)
print(male_count)
print(female_count)